In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, \
    HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.schema import SystemMessage
from langchain.chat_models import ChatOpenAI
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path

from htools import *
from jabberwocky.openai_utils import *

Object loaded from /Users/hmamin/jabberwocky/data/misc/gooseai_sample_responses.pkl.


In [3]:
cd_root()

Current directory: /Users/hmamin/roboduck


In [4]:
os.environ['OPENAI_API_KEY'] = api_key = load_openai_api_key()

In [5]:
print(load_prompt('debug')['prompt'])

debug: Could try davinci text as well but codex is free for now. You may want to strip triple double-quotes from the end in case codex generates them (we don't include that as a stop phrase because codex might generate a docstring as part of a correct code snippet).
-------------------------------------------------------------------------------

"""ANSWER KEY

This code snippet is not working as expected. Help me debug it. First read my question, then examine the snippet of code that is causing the issue and look at the values of the local and global variables. In the section titled SOLUTION PART 1, use plain English to explain what the problem is and how to fix it. In the section titled SOLUTION PART 2, write a corrected version of the input code snippet. If you don't know what the problem is, SOLUTION PART 1 should list a few possible causes or things I could try in order to identify the issue and SOLUTION PART 2 should say N/A. Be concise and use simple language because I am a begin

In [6]:
system_prompt_text = """You are an incredibly effective AI programming assistant. You have in-depth knowledge across a broad range of sub-fields within computer science, software development, and data science, and your goal is to help Python programmers resolve their most challenging bugs.
"""
system_prompt = SystemMessage(content=system_prompt_text)

In [7]:
user_prompt_text = """This code snippet is not working as expected. Help me debug it. First read my question, then examine the snippet of code that is causing the issue and look at the values of the local and global variables. Your response must have exactly two parts. In the section titled SOLUTION PART 1, use plain English to explain what the problem is and how to fix it (if you don't know what the problem is, SOLUTION PART 1 should instead list a few possible causes or things I could try in order to identify the issue). In the section titled SOLUTION PART 2, write a corrected version of the input code snippet (if you don't know, SOLUTION PART 2 should say None). SOLUTION PART 2 must contain only python code - there must not be any English explanation outside of code comments or docstrings. Be concise and use simple language because I am a beginning programmer.

QUESTION:
{question}

CURRENT CODE SNIPPET:
{code}

LOCAL VARIABLES:
{local_vars}

GLOBAL VARIABLES:
{global_vars}"""
user_prompt_template = HumanMessagePromptTemplate.from_template(
    user_prompt_text
)

In [8]:
kwargs = {
    'question': 'Why will this throw an index error soon?',
    'code': """def bubble_sort(nums):
    for i in range(len(nums)):
        for j in range(len(nums)):
            if nums[j] > nums[j + 1]:
                nums[j + 1], nums[j] = nums[j], nums[j + 1]
    return nums""",
    'local_vars': """{
    'nums': [3, 4, 2, 1, 5, 9],   # type: list
    'i': 0,   # type: int
    'j': 4,   # type: int
}""",
    'global_vars': """{
}"""
}

In [9]:
messages = [
    system_prompt,
    user_prompt_template.format(**kwargs)
]

In [10]:
print('\n'.join(m.content for m in messages))

You are an incredibly effective AI programming assistant. You have in-depth knowledge across a broad range of sub-fields within computer science, software development, and data science, and your goal is to help Python programmers resolve their most challenging bugs.

This code snippet is not working as expected. Help me debug it. First read my question, then examine the snippet of code that is causing the issue and look at the values of the local and global variables. Your response must have exactly two parts. In the section titled SOLUTION PART 1, use plain English to explain what the problem is and how to fix it (if you don't know what the problem is, SOLUTION PART 1 should instead list a few possible causes or things I could try in order to identify the issue). In the section titled SOLUTION PART 2, write a corrected version of the input code snippet (if you don't know, SOLUTION PART 2 should say None). SOLUTION PART 2 must contain only python code - there must not be any English ex

In [195]:
chat = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [58]:
res = chat(messages)
print(res.content)

SOLUTION PART 1:
The code will throw an index error soon because the inner loop is iterating up to the length of the list, which means that on the last iteration, `nums[j + 1]` will be out of range. To fix this, we need to change the range of the inner loop to `range(len(nums) - i - 1)`.

SOLUTION PART 2:

```
def bubble_sort(nums):
    for i in range(len(nums)):
        for j in range(len(nums) - i - 1):
            if nums[j] > nums[j + 1]:
                nums[j + 1], nums[j] = nums[j], nums[j + 1]
    return nums
```


In [60]:
messages.append(res)
messages.append(
    user_prompt_template.format(
        **{**kwargs, 
           'question': 'Can you revise your solution so you only find the length of nums once?'}
    )
)

In [62]:
res = chat(messages)
print(res.content)

SOLUTION PART 1:
The problem with the current code is that it is calling `len(nums)` twice in the inner loop, which is inefficient. To fix this, we can store the length of `nums` in a variable before the loop and use that variable instead.

SOLUTION PART 2:

```
def bubble_sort(nums):
    n = len(nums)
    for i in range(n):
        for j in range(n - i - 1):
            if nums[j] > nums[j + 1]:
                nums[j + 1], nums[j] = nums[j], nums[j + 1]
    return nums
```


Took first stab at storing this info in a file (py for now). Try loading.

In [11]:
import importlib
from langchain.schema import AIMessage
from roboduck.prompts.chat import debug

In [39]:
ChatOpenAI.__call__??

In [12]:
class DummyChatModel:
    
    def __init__(self, **kwargs):
        self.kwargs = kwargs
        
    def __call__(self, messages, stop=None):
        return AIMessage(content=messages[-1].content.upper())

In [27]:
class Chat:
    
    # Completion kwarg names that langchain requires you to pass in separately
    # from other model_kwargs when creating ChatOpenAI object (or similar).
    # Don't include kwargs like 'streaming' or 'n' here,
    # which shouldn't really be specified at the config level in most cases.
    # Not sure how consistent this interface will be for non-openai models but
    # I guess I'm putting my faith in langchain so might as well embrace it.
    _non_model_kwargs = {'max_tokens'}
    
    def __init__(self, system_text, user_text, model_name='gpt-3.5-turbo',
                 model_kwargs=None, chat_kwargs=None, chat_class=ChatOpenAI,
                 history=()):
        # chat_kwargs are those we unpack in our chat_class instantiation.
        # model_kwargs are the model_kwargs arg we pass to that instantiation,
        # not unpacked. E.g. chat_kwargs contains things like 
        # `callback_manager` or `verbose`, model_kwargs contains things like
        # `temperature` or `top_p`.
        self.model_kwargs, self.chat_kwargs = self._separate_chat_kwargs(
            model_kwargs, chat_kwargs
        )
        self.chat = chat_class(**self.chat_kwargs, 
                               model_kwargs=self.model_kwargs)
        self.system_message = SystemMessage(content=system_text)
        if isinstance(user_text, str):
            user_text = {'reply': user_text}
        self.user_templates = {
            k: HumanMessagePromptTemplate.from_template(v)
            for k, v in user_text.items()
        }
        self.default_user_key = next(iter(self.user_templates))
        self._history = list(history) or [self.system_message]
        
    @classmethod
    def load_template(cls, prompt_name, **extra_kwargs):
        # TODO: maybe let user modify defaults and/or pass in other init 
        # kwargs like verbose=False, ideally without copying signature
        # defaults from init manual (hard to maintain).
        # TODO: maybe just import prompts.chat upfront.
        module = importlib.import_module(
            f'roboduck.prompts.chat.{prompt_name}'
        )
        kwargs = {
            'system_text': module.system, 
            'user_text': module.user, 
            'model_name': module.model_name, 
            'model_kwargs': module.kwargs,
        }
        kwargs.update(extra_kwargs)
        return cls(**kwargs)
    
    def _separate_chat_kwargs(self, model_kwargs=None, chat_kwargs=None):
        chat_kwargs = dict(chat_kwargs or {})
        model_kwargs = dict(model_kwargs or {})
        keys_to_move = set(model_kwargs) & self._non_model_kwargs
        for key in keys_to_move:
            chat_kwargs[key] = model_kwargs.pop(key)
        return model_kwargs, chat_kwargs
        
    def _user_message(self, *, key='', **kwargs):
        key = key or self.default_user_key
        template = self.user_templates[key]
        return template.format(**kwargs)
    
    def reply(self, *, key='', **kwargs):
        user_message = self._user_message(key=key, **kwargs)
        self._history.append(user_message)
        try:
            response = self.chat(self._history)
        except Exception as e:
            self._history.pop(-1)
            raise e
        self._history.append(response)
        return response
    
    def history(self, sep='\n\n', speaker_prefix=True):
        """Return chat history as a single string."""
        res = []
        for row in self._history:
            reply = row.content
            if speaker_prefix:
                speaker = type(row).__name__.split('Message')[0]
                reply = f'{speaker}: {reply}'
            res.append(reply)
        return sep.join(res)

In [28]:
chat = Chat.load_template('debug', chat_class=DummyChatModel)

In [29]:
chat.chat

In [30]:
tmp = chat._user_message(
    code='a = 3\nb = 4', question='Why?', local_vars='{3: 4}',
    global_vars='{True: False}', next_line='b = 4'
)

In [31]:
print(tmp.content)

This code snippet is not working as expected. Help me debug it. First read my question, then examine the snippet of code that is causing the issue and look at the values of the local and global variables. Your response must have exactly two parts. In the section titled SOLUTION PART 1, use plain English to explain what the problem is and how to fix it (if you don't know what the problem is, SOLUTION PART 1 should instead list a few possible causes or things I could try in order to identify the issue). In the section titled SOLUTION PART 2, write a corrected version of the input code snippet (if you don't know, SOLUTION PART 2 should say None). SOLUTION PART 2 must contain only python code - there must not be any English explanation outside of code comments or docstrings. Be concise and use simple language because I am a beginning programmer.

QUESTION:
Why?

CURRENT CODE SNIPPET:
a = 3
b = 4

NEXT LINE:
b = 4

LOCAL VARIABLES:
{3: 4}

GLOBAL VARIABLES:
{True: False}


In [32]:
tmp = chat._user_message(
    key='contextless',
    question='Why?'
)

In [33]:
print(tmp.content)

QUESTION:
Why?


In [34]:
chat.chat

In [35]:
chat.reply(key='contextless', question='How are you?')

AIMessage(content='QUESTION:\nHOW ARE YOU?', additional_kwargs={})

In [36]:
print(chat.history())

System: You are an incredibly effective AI programming assistant. You have in-depth knowledge across a broad range of sub-fields within computer science, software development, and data science, and your goal is to help Python programmers resolve their most challenging bugs.

Human: QUESTION:
How are you?

AI: QUESTION:
HOW ARE YOU?


In [38]:
# This would make more sense if our messages included speakers, i.e. if a
# user_message looked like 'Me: {reply}' and gpt was prompted to reply like
# 'Robert Sapolsky: {reply}' (for example).
print(chat.history(speaker_prefix=False))

You are an incredibly effective AI programming assistant. You have in-depth knowledge across a broad range of sub-fields within computer science, software development, and data science, and your goal is to help Python programmers resolve their most challenging bugs.

QUESTION:
How are you?

QUESTION:
HOW ARE YOU?


Error when using real ChatOpenAI obj.

In [24]:
chat = Chat.load_template('debug')
chat.chat

ChatOpenAI(verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7fed643756d0>, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', model_kwargs={'temperature': 0.0, 'top_p': 0.99, 'frequency_penalty': 0.2, 'presence_penalty': 0.0, 'logit_bias': {37811: -100, 27901: -50}, 'stop': ['QUESTION', 'SOLUTION PART 1', 'SOLUTION PART 2']}, openai_api_key=None, request_timeout=60, max_retries=6, streaming=False, n=1, max_tokens=512)

In [25]:
chat.reply(key='contextless', question='How are you?')

AIMessage(content="As an AI language model, I don't have feelings, but I'm functioning properly and ready to assist you with any programming-related questions you may have. How can I help you today?", additional_kwargs={})

In [26]:
print(chat.history())

System: You are an incredibly effective AI programming assistant. You have in-depth knowledge across a broad range of sub-fields within computer science, software development, and data science, and your goal is to help Python programmers resolve their most challenging bugs.

Human: QUESTION:
How are you?

AI: As an AI language model, I don't have feelings, but I'm functioning properly and ready to assist you with any programming-related questions you may have. How can I help you today?


## Debug scratch

See if we can use frames to identify whether we need to provide context for a user message (i.e. if frame has changed since we last did).

In [4]:
from roboduck.debugger import duck

In [70]:
def binary_search(x, nums):
    if not nums:
        return -1
    duck(backend='repeat')
    mid = len(nums) // 2
    if x == nums[mid]:
        return x
    if x > nums[mid]:
        return binary_search(x, nums[mid + 1:])
    if x < nums[mid]:
        return binary_search(x, nums[:mid])

In [71]:
nums = [33, 44, 55, 66, 77, 88, 99, 111]

In [72]:
binary_search(3, nums)

> <ipython-input-70-8a37149461d4>(5)binary_search()
-> mid = len(nums) // 2
>>> l .
  1  	def binary_search(x, nums):
  2  	    if not nums:
  3  	        return -1
  4  	    duck(backend='repeat')
  5  ->	    mid = len(nums) // 2
  6  	    if x == nums[mid]:
  7  	        return x
  8  	    if x > nums[mid]:
  9  	        return binary_search(x, nums[mid + 1:])
 10  	    if x < nums[mid]:
 11  	        return binary_search(x, nums[:mid])
>>> y?
next line:     mid = len(nums) // 2
[Duck] """>>> n
> <ipython-input-70-8a37149461d4>(6)binary_search()
-> if x == nums[mid]:
>>> n
> <ipython-input-70-8a37149461d4>(8)binary_search()
-> if x > nums[mid]:
>>> n
> <ipython-input-70-8a37149461d4>(10)binary_search()
-> if x < nums[mid]:
>>> n
> <ipython-input-70-8a37149461d4>(11)binary_search()
-> return binary_search(x, nums[:mid])
>>> l .
  6  	    if x == nums[mid]:
  7  	        return x
  8  	    if x > nums[mid]:
  9  	        return binary_search(x, nums[mid + 1:])
 10  	    if x < nums[mid

BdbQuit: 

In [13]:
binary_search(33, nums)

33

In [14]:
binary_search(39, nums)

-1

In [15]:
binary_search(111, nums)

111

In [16]:
binary_search(112, nums)

-1

In [73]:
def test():
    for i in range(5):
        print(i)
        duck(backend='repeat')

In [74]:
test()

0
> <ipython-input-73-943befe6b744>(2)test()
-> for i in range(5):
>>> i
0
>>> l .
  1  	def test():
  2  ->	    for i in range(5):
  3  	        print(i)
  4  	        duck(backend='repeat')
[EOF]
>>> y?
frmae_id 140192236901280
next line:     for i in range(5):
[Duck] """>>> n
> <ipython-input-73-943befe6b744>(3)test()
-> print(i)
>>> i
1
>>> y?
frmae_id 140192236901280
next line:         print(i)
[Duck] """>>> n
1
> <ipython-input-73-943befe6b744>(4)test()
-> duck(backend='repeat')
>>> i
1
>>> n
> <ipython-input-73-943befe6b744>(2)test()
-> for i in range(5):
>>> i
1
>>> l .
  1  	def test():
  2  ->	    for i in range(5):
  3  	        print(i)
  4  	        duck(backend='repeat')
[EOF]
>>> i
1
>>> y?
frmae_id 140192236901280
next line:     for i in range(5):
[Duck] """>>> n
> <ipython-input-73-943befe6b744>(3)test()
-> print(i)
>>> l .
  1  	def test():
  2  	    for i in range(5):
  3  ->	        print(i)
  4  	        duck(backend='repeat')
[EOF]
>>> y?
frmae_id 140192236901280


BdbQuit: 